In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50

In [ ]:
from tensorflow.keras import datasets, layers, models

from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D
from tensorflow.keras import Input
from tensorflow.keras.layers import Dropout, BatchNormalization

import matplotlib.pyplot as plt

In [ ]:
# 케라스 데이터셋을 다운받아 변수에 각각 넣어준다.
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

170498071/170498071 [==============================] - 6s 0us/step


In [ ]:
train_images.shape, train_labels.shape, test_images.shape, test_labels.shape

((50000, 32, 32, 3), (50000, 1), (10000, 32, 32, 3), (10000, 1))

In [ ]:
# 라벨 설정
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

## 전이학습 (transfer learning)

### resnet50_ver1

In [ ]:
resnet_v1 = ResNet50(include_top=False, input_shape = (32,32 ,3), weights = 'imagenet')

94765736/94765736 [==============================] - 3s 0us/step


In [ ]:
resnet_v1.output_shape

(None, 1, 1, 2048)

1) include_top = True, 분류기(완전연결계층) 여부  
2) weights = 'imagenet', 사전학습 weight 여부  
3) input_shape, Input 사이즈 (32, 32, 3)으로 변경

In [ ]:
resnet_v1.trainable = False

In [ ]:
# sparse categorical crossentropy VS categorical crossentropy + one_hot 
y_train = tf.keras.utils.to_categorical(train_labels, 10)
y_test = tf.keras.utils.to_categorical(test_labels, 10)
y_train.shape, y_test.shape

((50000, 10), (10000, 10))

In [ ]:
y_train[0]

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.], dtype=float32)

In [ ]:
# model.Sequential()없이 모델 구축하는 방법
inputs = tf.keras.Input(shape=(32, 32, 3))

x = resnet_v1(inputs, training=False)
x = tf.keras.layers.Flatten(input_shape=resnet_v1.output_shape[1:])(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x= tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(10, activation='softmax')(x)

model_v1 = tf.keras.Model(inputs, outputs)

In [ ]:
# categorical_crossentropy
model_v1.compile(optimizer = tf.keras.optimizers.Adam(learning_rate= 0.001),
                  loss = 'categorical_crossentropy',
                  metrics=['accuracy'])

# 모델 fitting
model_v1.fit(train_images, y_train, epochs = 1, validation_data=(test_images, y_test), batch_size=128)

391/391 [==============================] - 175s 440ms/step - loss: 1.5902 - accuracy: 0.4653 - val_loss: 1.2311 - val_accuracy: 0.5663


In [ ]:
pred_v1 = model_v1.predict(test_images)

313/313 [==============================] - 35s 108ms/step


In [ ]:
pred_v1

array([[1.07208837e-03, 2.62347725e-03, 3.83071192e-02, ...,
        1.14391446e-02, 1.61645783e-03, 9.72185167e-04],
       [7.80295432e-02, 4.34799999e-01, 5.13904524e-05, ...,
        4.33954046e-05, 3.86061549e-01, 1.00566402e-01],
       [2.60466971e-02, 1.84141267e-02, 7.06946375e-05, ...,
        1.33079884e-04, 9.12598073e-01, 4.02721465e-02],
       ...,
       [1.14544215e-04, 8.22945847e-04, 6.14960305e-02, ...,
        1.81575101e-02, 1.34818896e-04, 3.96248543e-05],
       [5.23066744e-02, 8.86401460e-02, 8.21480006e-02, ...,
        2.88659096e-01, 2.59092972e-02, 7.09504932e-02],
       [1.80139148e-04, 7.63621865e-05, 1.01072912e-03, ...,
        9.61545169e-01, 4.87713442e-06, 1.29761989e-04]], dtype=float32)

### resnet_v2

In [ ]:
resnet_v2 = ResNet50(include_top=False, input_shape = (32,32 ,3), weights = 'imagenet')

In [ ]:
resnet_v2.output_shape

(None, 1, 1, 2048)

1) include_top = True, 분류기(완전연결계층) 여부  
2) weights = 'imagenet', 사전학습 weight 여부  
3) input_shape, Input 사이즈 (32, 32, 3)으로 변경

In [ ]:
resnet_v2.trainable = False

In [ ]:
# sparse categorical crossentropy VS categorical crossentropy + one_hot 
y_train = tf.keras.utils.to_categorical(train_labels, 10)
y_test = tf.keras.utils.to_categorical(test_labels, 10)
y_train.shape, y_test.shape

((50000, 10), (10000, 10))

In [ ]:
y_train[0]

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.], dtype=float32)

In [ ]:
# model.Sequential()없이 모델 구축하는 방법
inputs = tf.keras.Input(shape=(32, 32, 3))

x = resnet_v1(inputs, training=False)
x = tf.keras.layers.Flatten(input_shape=resnet_v2.output_shape[1:])(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x= tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(10, activation='softmax')(x)

model_v2 = tf.keras.Model(inputs, outputs)

In [ ]:
# categorical_crossentropy
model_v2.compile(optimizer = tf.keras.optimizers.Adam(learning_rate= 0.001),
                  loss = 'categorical_crossentropy',
                  metrics=['accuracy'])

# 모델 fitting
model_v2.fit(train_images, y_train, epochs = 1, validation_data=(test_images, y_test), batch_size=128)

391/391 [==============================] - 191s 480ms/step - loss: 1.5271 - accuracy: 0.5009 - val_loss: 1.1589 - val_accuracy: 0.5942


In [ ]:
pred_v2 = model_v2.predict(test_images)

313/313 [==============================] - 37s 116ms/step


In [ ]:
pred_v2

array([[9.2175566e-03, 2.3053359e-02, 6.9628678e-02, ..., 1.3646538e-02,
        9.4250552e-03, 3.3973830e-03],
       [2.8464772e-02, 6.6411746e-01, 3.1961667e-05, ..., 1.8805087e-05,
        2.7734149e-01, 2.9771727e-02],
       [1.9429367e-02, 5.2799736e-03, 1.0584139e-04, ..., 3.6090397e-04,
        9.2794478e-01, 4.6099767e-02],
       ...,
       [9.8564495e-05, 1.0036316e-03, 5.8076769e-02, ..., 2.5826663e-02,
        1.2673408e-04, 9.4720459e-04],
       [9.0584040e-02, 1.0179939e-01, 1.8107086e-01, ..., 1.4410292e-01,
        4.4423774e-02, 3.8907010e-02],
       [1.5860028e-04, 8.7390901e-05, 5.4661916e-03, ..., 8.7391907e-01,
        1.8264975e-05, 6.8890811e-05]], dtype=float32)

### 앙상블

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score

In [ ]:
pred_ensemble = np.mean([pred_v1, pred_v2], axis=0)
pred_ensemble

array([[5.14482241e-03, 1.28384186e-02, 5.39679006e-02, ...,
        1.25428420e-02, 5.52075636e-03, 2.18478404e-03],
       [5.32471575e-02, 5.49458742e-01, 4.16760595e-05, ...,
        3.11002441e-05, 3.31701517e-01, 6.51690662e-02],
       [2.27380320e-02, 1.18470499e-02, 8.82680179e-05, ...,
        2.46991927e-04, 9.20271397e-01, 4.31859568e-02],
       ...,
       [1.06554355e-04, 9.13288735e-04, 5.97864017e-02, ...,
        2.19920874e-02, 1.30776491e-04, 4.93414700e-04],
       [7.14453608e-02, 9.52197686e-02, 1.31609440e-01, ...,
        2.16381013e-01, 3.51665355e-02, 5.49287498e-02],
       [1.69369712e-04, 8.18765402e-05, 3.23846028e-03, ...,
        9.17732120e-01, 1.15710545e-05, 9.93263966e-05]], dtype=float32)

In [ ]:
pred_ensemble = np.argmax(pred_ensemble, axis=1)

In [ ]:
y_test = np.argmax(y_test, axis=1)

In [ ]:
y_test.shape, pred_ensemble.shape

((10000,), (10000,))

In [ ]:
accuracy_score(pred_ensemble, y_test)

0.5997

In [ ]:
# 단일 모델과 비교
accuracy_score(np.argmax(pred_v1, axis=1), y_test)

0.5663

In [ ]:
accuracy_score(np.argmax(pred_v2, axis=1), y_test)

0.5942